In [3]:
!pip install -q datasets
!pip install -U transformers --upgrade
!pip install trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 15.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-ma

In [ ]:
!pip install bitsandbytes

In [20]:
!pip install accelerate -U
!pip install transformers[torch]

In [1]:
import os
from dataclasses import dataclass, field
from typing import Dict, Optional

import torch
from datasets import Dataset, load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, HfArgumentParser, TrainingArguments, T5Config, T5Tokenizer, T5ForConditionalGeneration

from trl import DPOTrainer



In [2]:
model_id = 'google-t5/t5-small'

In [3]:
config = T5Config.from_pretrained(model_id)

tokenizer = T5Tokenizer.from_pretrained(model_id)

model = T5ForConditionalGeneration.from_pretrained(model_id,config=config)
model.config.use_cache = False

model_ref = T5ForConditionalGeneration.from_pretrained(model_id,config=config)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
device = "cuda"

In [5]:
model = model.to(device)
model_ref = model_ref.to(device)

In [6]:
def return_prompt_and_responses(samples) -> Dict[str, str]:
        return {
            "prompt": samples["queries"],
            "chosen": samples["labels"],
            "rejected": samples["predictions"],
        }



In [7]:
train_dataset = load_dataset("json", data_files={"train" : "/content/loose_EM_t5_results.json"})

In [9]:
train_data = train_dataset['train'].select([i for i in range(0,500)])
eval_data = train_dataset['train'].select([i for i in range(500,600)])
test_data = train_dataset['train'].select([i for i in range(800,1000)])

In [10]:
original_columns = train_data.column_names
# dataset = dataset.select(range(min(len(dataset), 500)))
train_data =  train_data.map(
    return_prompt_and_responses,
    batched=True,
    remove_columns=original_columns,
)
eval_data =  eval_data.map(
    return_prompt_and_responses,
    batched=True,
    remove_columns=original_columns,
)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [11]:
OUTPUT_DIR = "llama2-docsum-adapter2"
training_args = TrainingArguments(
    per_device_train_batch_size=12,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    logging_steps=1,
    learning_rate=2e-5,
    # fp16=True,
    max_grad_norm=0.2,
    num_train_epochs=3,
    evaluation_strategy="steps",
    eval_steps=0.2,
    warmup_ratio=0.05,
    save_strategy="epoch",
    # group_by_length=True,
    output_dir=OUTPUT_DIR,
    report_to="tensorboard",
    save_safetensors=True,
    lr_scheduler_type="cosine",
    seed=42,
    remove_unused_columns=False,
    run_name="dpo"
)

In [12]:

dpo_trainer = DPOTrainer(
    model,
    model_ref,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
    tokenizer=tokenizer,
    beta = 0.1
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:300: UserWarning: `max_length` is not set in the DPOTrainer's init it will default to `512` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:307: UserWarning: `max_prompt_length` is not set in the DPOTrainer's init it will default to `128` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:315: UserWarning: When using an encoder decoder architecture, you should set `max_target_length` in the DPOTrainer's init it will default to `128` by default, but you should do it yourself in the future.
  warnings.warn(


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [13]:
dpo_trainer.train()

Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/rejected,Logps/chosen,Logits/rejected,Logits/chosen
6,0.706100,0.660206,0.122358,0.052212,0.740385,0.070145,-26.367119,-22.957956,-22.115831,-20.531420
12,0.654400,0.629008,0.205562,0.061567,0.769231,0.143995,-26.273573,-22.125914,-22.328550,-20.791563
18,0.594000,0.610639,0.234912,0.045060,0.788462,0.189852,-26.438648,-21.832415,-22.518419,-21.024973
24,0.646500,0.603765,0.242847,0.034882,0.798077,0.207965,-26.540419,-21.753061,-22.618900,-21.147198
30,0.616600,0.602441,0.243605,0.032187,0.798077,0.211418,-26.567366,-21.745483,-22.632664,-21.166492


TrainOutput(global_step=30, training_loss=0.6429237425327301, metrics={'train_runtime': 46.6944, 'train_samples_per_second': 32.124, 'train_steps_per_second': 0.642, 'total_flos': 0.0, 'train_loss': 0.6429237425327301, 'epoch': 2.857142857142857})

In [14]:
test_data[0]

{'labels': '110 mph',
 'EM': 0.0,
 'average_score': 0.6681072506,
 'loose_EM': 0.0,
 'queries': "Question: How fast were the winds around St. Augustine in the 1964 hurricane?;Context: jacksonville has suffered less damage from hurricanes than most other east coast cities, although the threat does exist for a direct hit by a major hurricane. the city has only received one direct hit from a hurricane since 1871; however, jacksonville has experienced hurricane or near-hurricane conditions more than a dozen times due to storms crossing the state from the gulf of mexico to the atlantic ocean, or passing to the north or south in the atlantic and brushing past the area. the strongest effect on jacksonville was from hurricane dora in 1964, the only recorded storm to hit the first coast with sustained hurricane-force winds. the eye crossed st. augustine with winds that had just barely diminished to 110 mph (180 km/h), making it a strong category 2 on the saffir-simpson scale. jacksonville also 

In [19]:
input_ids = tokenizer(test_data[0]['queries'], return_tensors="pt").to(device)
generator_output = dpo_trainer.model.generate(**input_ids)


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [20]:
decoded_generator_output = tokenizer.decode(generator_output[0], skip_special_tokens=True)

In [21]:
decoded_generator_output

'jacksonville has suffered less damage from hurricanes than most other east coast cities.'

In [ ]:
dpo_trainer.model_ref.